# advent of code 2024 - [day 16](https://adventofcode.com/2024/day/16)

In [243]:
import os

NEO4J_URI = os.environ['NEO4J_URI']
NEO4J_USERNAME = os.environ['NEO4J_USERNAME']
NEO4J_PASSWORD = os.environ['NEO4J_PASSWORD']


In [244]:
from graphdatascience import GraphDataScience
gds = GraphDataScience(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

# Check the installed GDS version on the server
print(gds.version())
assert gds.version()

from neo4j import GraphDatabase
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

import pandas as pd

2.13.0


In [245]:
queries = [
"CALL apoc.schema.assert({},{});",
"""MATCH (n)
CALL {WITH n DETACH DELETE n}
IN TRANSACTIONS OF 1000 ROWS;""",
"""CALL gds.graph.list()
YIELD graphName
WITH graphName AS g
CALL (g) {CALL gds.graph.drop(g) YIELD graphName RETURN graphName}
WITH graphName RETURN graphName;"""]

for q in queries:
    gds.run_cypher(q, {})


In [246]:
def gen_lists(file='input.txt'):
    """Generates tuples of integers"""
    file = open(file, 'r')
    for ix, line in enumerate(file):
        for jx, c in enumerate(line.strip()):
            yield ix, jx, c

In [247]:
filename = "input.txt"
filename = "test.txt"


## Neo4j-based solution

### Parsing

In [248]:
tiles = [{'row':ix, 'col':jx, 'val':c} for ix, jx, c in list(gen_lists(filename))]

### Building Lattice

In [250]:
gds.run_cypher('CREATE INDEX tile_row IF NOT EXISTS FOR (r:Tile) ON (r.row)')
gds.run_cypher('CREATE INDEX tile_col IF NOT EXISTS FOR (r:Tile) ON (r.col)')
gds.run_cypher('CREATE INDEX tile_val IF NOT EXISTS FOR (r:Tile) ON (r.val)')
gds.run_cypher('CREATE INDEX port_comp IF NOT EXISTS FOR (p:Port) ON (p.row, p.col, p.dir)')
gds.run_cypher('CREATE INDEX has_port_dir IF NOT EXISTS FOR ()-[r:HAS_PORT]-() ON (r.dir)')


""


In [251]:
query_ingest = """
UNWIND $tiles AS tile
CREATE (:Tile {row:tile.row, col:tile.col, val:tile.val} )
"""

gds.run_cypher(query_ingest, {"tiles":tiles})

""


In [252]:
gds.run_cypher("""
MATCH (t:Tile)
WITH t.row AS row_num, t ORDER BY t.col
WITH row_num, collect(t) AS row
CALL apoc.nodes.link(row, 'EAST')
""")

""


In [253]:
gds.run_cypher("""
MATCH (t:Tile)
WITH t.col AS col_num, t ORDER BY t.row
WITH col_num, collect(t) AS col
CALL apoc.nodes.link(col, 'SOUTH')
""")

""


In [254]:
gds.run_cypher("""
MATCH (x)-[:EAST]->(y)
CALL (x,y) {
MERGE (x)-[:NEXT_TO {dir: '>'}]->(y)
MERGE (y)-[:NEXT_TO {dir: '<'}]->(x)
} IN TRANSACTIONS OF 1000 ROWS
""")

gds.run_cypher("""
MATCH (x)-[:SOUTH]->(y)
CALL (x,y) {
MERGE (x)-[:NEXT_TO {dir: 'v'}]->(y)
MERGE (y)-[:NEXT_TO {dir: '^'}]->(x)
} IN TRANSACTIONS OF 1000 ROWS
""")

""


In [255]:
gds.run_cypher("""
MATCH (t:Tile)
CALL (t){
UNWIND ['v','<','^','>'] AS dir
MERGE (p:Port{row:t.row, col:t.col, dir:dir})
    SET p.val=t.val
MERGE  (t)-[:HAS_PORT {dir:dir}]->(p)
} IN CONCURRENT TRANSACTIONS OF 1000 ROWS
""")

""


In [256]:
gds.run_cypher("""
MATCH (t1:Tile)-[r:NEXT_TO]->(t2:Tile)
CALL (t1, r, t2) {
MATCH (t1)-[:HAS_PORT {dir: r.dir}]->(p1), (t2)-[:HAS_PORT {dir: r.dir}]->(p2)
MERGE (p1)-[:POSSIBLE_MOVE {cost: 1}]->(p2)
} IN TRANSACTIONS OF 100 ROWS
""")

gds.run_cypher("""
MATCH (t:Tile)
CALL (t) {
MATCH (t)-[r:HAS_PORT]->(p1), (t)-[:HAS_PORT {dir: {`v`:'<', `<`:'^', `^`:'>', `>`:'v'}[r.dir]}]->(p2)
MERGE (p1)-[:POSSIBLE_MOVE {cost:1000}]->(p2)
MERGE (p2)-[:POSSIBLE_MOVE {cost:1000}]->(p1)
} IN TRANSACTIONS OF 100 ROWS
""")

gds.run_cypher("""
MATCH (t:Port {val:'E'})
MERGE(target:Port:Target)
MERGE (t)-[:POSSIBLE_MOVE {cost:0}]->(target)
""")

gds.run_cypher("""
MATCH (t:Port {val:'S', dir:'>'})
MERGE(start:Port:Start)
MERGE (start)-[:POSSIBLE_MOVE {cost:0}]->(t)
""")


""


In [257]:
gds.run_cypher("""
MATCH (p1:Port WHERE p1.val IS NULL OR p1.val <> '#')
OPTIONAL MATCH (p1)-[r:POSSIBLE_MOVE]->(p2:Port WHERE p2.val IS NULL OR p2.val <> '#')
RETURN gds.graph.project(
  'maze',
  p1,
  p2,
  { relationshipProperties: r { .cost } }
)
""")


,"gds.graph.project(\n 'maze',\n p1,\n p2,\n { relationshipProperties: r { .cost } }\n)"
0,"{'relationshipCount': 1057, 'graphName': 'maze..."


### Solving part 1

In [258]:
gds.run_cypher("""
MATCH (source:Port:Start), (target:Port:Target)
CALL gds.shortestPath.dijkstra.stream('maze', {
    sourceNode: source,
    targetNodes: target,
    relationshipWeightProperty: 'cost'
})
YIELD totalCost
RETURN
    toInteger(totalCost) AS part1
""")

,part1
0,7036


In [ ]:
gds.run_cypher("""
MATCH (source:Start),(target:Target)
MATCH (t:Port&!(Start|Target) WHERE t.val IN ['.', 'S', 'E'])
CALL (source, target, t) {
  CALL gds.shortestPath.dijkstra.stream('maze', {
    sourceNode: source,
    targetNodes: t,
    relationshipWeightProperty: 'cost'
  })
  YIELD totalCost AS leg_1_cost
  WITH source, target, t, leg_1_cost
  CALL gds.shortestPath.dijkstra.stream('maze', {
    sourceNode: t,
    targetNodes: target,
    relationshipWeightProperty: 'cost'
  })
  YIELD totalCost AS leg_2_cost
  RETURN leg_1_cost, leg_2_cost
} IN CONCURRENT TRANSACTIONS OF 500 ROWS
WITH t, leg_1_cost, leg_2_cost, leg_1_cost + leg_2_cost AS total ORDER BY total          
MATCH (t)<-[:HAS_PORT]-(tile:Tile)
WITH tile, min(total) AS total
RETURN total, count(*) AS part2 
""")

,total,part2
0,7036.0,45
1,9038.0,4
2,9040.0,4
3,10028.0,13
4,11038.0,1
5,11040.0,1
6,11042.0,3
7,11044.0,3
8,11046.0,2
9,11048.0,5


## viz

In [260]:
gds.run_cypher("MATCH (t:Tile) SET t.X = t.col, t.Y = t.row")
gds.run_cypher("""MATCH (t:Tile)-[r:HAS_PORT]->(p)
  SET p.X = t.X + {`v`:0.0, `<`:-0.3, `^`:0, `>`:0.3}[r.dir],
  p.Y = 1000 - (t.Y + {`v`:0.3, `<`:0.0, `^`:-0.3, `>`:0.0}[r.dir])""")

""
